In [41]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [42]:
x_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')
x_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

In [43]:
x_test.shape

(654, 224, 224, 3)

In [44]:
#x_train /= 255????

In [45]:
from __future__ import print_function

import numpy as np
from random import shuffle
import time
import csv
from PIL import Image
import os
import tensorflow as tf
import keras
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import initializers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import *
from keras.callbacks import CSVLogger
from keras import callbacks
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklm
import lossprettifier
from ResNet import *

In [46]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 
  
# One hot encoding of labels 
def dense_to_one_hot(labels_dense,num_clases=4):
  return np.eye(num_clases)[labels_dense]

In [47]:
# Preparing training and test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [48]:
y_train = dense_to_one_hot(y_train,num_clases=4)
y_valid= dense_to_one_hot(y_valid,num_clases=4)

In [49]:
#Image data generation for the training 
datagen = ImageDataGenerator(
               featurewise_center = False, 
               samplewise_center = False,  # set each sample mean to 0
               featurewise_std_normalization = True,  
               samplewise_std_normalization = False)  

datagen.fit(x_train) 
for i in range(len(x_test)):
      x_test[i] = datagen.standardize(x_test[i])

/home/reza/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:348: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
#Defining hyperparameters
batch_Size = 32
steps_Per_Epoch = 32
numEpochs = 50

#Instantating ResNet18 model
model = ResNet18((224, 224, 3), 4) 

#Creating an optimizers
adaDelta = keras.optimizers.Adadelta(lr=1.0, rho=0.95)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.95, nesterov=True)
model.compile(optimizer = sgd , loss = 'categorical_crossentropy', metrics = ['accuracy'])

#Creating early stopping 
earlystop = EarlyStopping(monitor = 'val_accuracy', min_delta = 0, patience = 50, verbose = 1, mode = 'auto', restore_best_weights = True)       

train_generator = datagen.flow(x_train, y_train, batch_size = batch_Size)
validation_generator = datagen.flow(x_valid, y_valid, batch_size = batch_Size)

# Model training
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_Per_Epoch,
    validation_data = validation_generator, 
    validation_steps = 16,
    epochs = numEpochs,
    shuffle = True, 
    verbose = 1)

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Epoch 1/50
32/32 [==============================] - 78s 2s/step - loss: 3.8593 - accuracy: 0.6270 - val_loss: 1130.2300 - val_accuracy: 0.2539
Epoch 2/50
32/32 [==============================] - 73s 2s/step - loss: 3.6147 - accuracy: 0.7143 - val_loss: 286.7065 - val_accuracy: 0.2704
Epoch 3/50
32/32 [==============================] - 74s 2s/step - loss: 3.5902 - accuracy: 0.7178 - val_loss: 102.1051 - val_accuracy: 0.2684
Epoch 4/50
32/32 [==============================] - 74s 2s/step - loss: 3.5250 - accuracy: 0.7393 - val_loss: 26.7099 - val_accuracy: 0.2505
Epoch 5/50
32/32 [==============================] - 75s 2s/step - loss: 3.4561 - accuracy: 0.7832 - val_loss: 14.2865 - val_accuracy: 0.2744
Epoch 6/50
32/32 [==============================] - 73s 2s/step - loss: 3.4043 - accuracy: 0.7695 - val_loss: 9.3615 - val_accuracy: 0.2704
Epoch 7/50
32/32 [=========

In [11]:
modelPath = "ResNet18_COVID19.h5"
resultPath = 'ResNet18_COVID19.txt'

In [29]:
# visualizing losses and accuracy
train_loss = history.history['loss']
val_loss = history.history['val_loss']

y_test_oh = dense_to_one_hot(y_test,num_clases=4)

#Observing the losses but can be commented out as it's not mandatory 
reporter = lossprettifier.LossPrettifier(show_percentage=True)

for i in range(numEpochs-1):
    reporter(epoch=i, LossA = train_loss[i], LossAB = val_loss[i])

# Model evaluation 
score, acc = model.evaluate(x_test, y_test_oh, batch_size=batch_Size)
print("Accuracy:", acc)

#if acc>0.675:
model.save_weights(modelPath)

Epoch     0 | LossA: 3.82(+0.00%) 	| LossAB: 1515.01(+0.00%) 	
Epoch     1 | LossA: 3.61(-5.36%) ▼	| LossAB: 293.95(-80.60%) ▼	
Epoch     2 | LossA: 3.59(-0.76%) ▼	| LossAB: 66.91(-77.24%) ▼	
Epoch     3 | LossA: 3.49(-2.55%) ▼	| LossAB: 29.26(-56.27%) ▼	
Epoch     4 | LossA: 3.48(-0.42%) ▼	| LossAB: 14.31(-51.10%) ▼	
Epoch     5 | LossA: 3.40(-2.45%) ▼	| LossAB: 5.35(-62.63%) ▼	
Epoch     6 | LossA: 3.42(+0.86%) ▲	| LossAB: 5.03(-5.97%) ▼	
Epoch     7 | LossA: 3.43(+0.21%) ▲	| LossAB: 7.25(+44.15%) ▲	
Epoch     8 | LossA: 3.41(-0.69%) ▼	| LossAB: 8.45(+16.64%) ▲	
Epoch     9 | LossA: 3.35(-1.62%) ▼	| LossAB: 3.71(-56.15%) ▼	
Epoch    10 | LossA: 3.32(-1.03%) ▼	| LossAB: 4.33(+16.90%) ▲	
Epoch    11 | LossA: 3.31(-0.14%) ▼	| LossAB: 4.06(-6.24%) ▼	
Epoch    12 | LossA: 3.31(-0.17%) ▼	| LossAB: 3.67(-9.78%) ▼	
Epoch    13 | LossA: 3.32(+0.32%) ▲	| LossAB: 3.74(+1.93%) ▲	
Epoch    14 | LossA: 3.21(-3.14%) ▼	| LossAB: 3.45(-7.80%) ▼	
Epoch    15 | LossA: 3.22(+0.32%) ▲	| LossAB: 3.31(-3.8

In [31]:
y_pred = model.predict(x_test)
#y_pred = y_pred.reshape(len(y_test), 4)
y_pred = np.argmax(y_pred, axis=1)

# Writing results on file
f = open(resultPath,'a') #create classification report
f.write(classification_report(y_test, y_pred))
f.write(str(sklm.cohen_kappa_score(y_test, y_pred))+","+str(acc)+","+str(score)+"\n")

#Print class-wise classification metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.59      0.73       234
           1       0.76      0.93      0.84       246
           2       0.59      0.70      0.64       149
           3       0.45      0.62      0.53         8

    accuracy                           0.75       637
   macro avg       0.69      0.71      0.68       637
weighted avg       0.79      0.75      0.75       637

